In [1]:
# Load librarys
library(edgeR)
library(limma)
library(Glimma)
library(ggplot2)
library(RColorBrewer)

Loading required package: limma



In [2]:
# Reading L3 count ordered data
allRawL3CountsData <- read.csv("/home/jupyter/clean_csv/raw_L3counts_data/raw_L3counts__ordered_data.csv", row.names=1)

In [3]:
# Reading groups like Sex, Ethnicity, etc
groupsList <- read.csv("/home/jupyter/clean_csv/raw_L3counts_data/transcript_data_groups.csv", row.names=1)
sex <- factor(groupsList$Sex)
Ethnicity <- factor(groupsList$Ethnicity)
Race <- factor(groupsList$Race)
SiteOnset <- factor(groupsList$SiteOnset)
age <- factor(groupsList$Age)

In [4]:
#Calculate count per million and keep samples
myCPM <- cpm(allRawL3CountsData)
thresh <- myCPM > 0.5
keep <- rowSums(thresh) >= 2
counts.keep <- allRawL3CountsData[keep,]
dge <- DGEList(counts=counts.keep)

In [85]:
#plot the library size per sample
barplot(dge$samples$lib.size, names=colnames(dge), las=2)

In [5]:
# Assign normalize data
nor.data <-normalizeQuantiles(counts.keep, ties=TRUE)
dge <- DGEList(nor.data)
heatmap(as.matrix(nor.data))

In [87]:
# Write quantile normalization csv file
write.csv(nor.data, "/home/jupyter/clean_csv/raw_L3counts_data/quantile_normalization.csv")

In [6]:
# To perform quasi-likelihood F-tests:
design <- model.matrix(~SiteOnset + Race + sex + age)
y <- estimateDisp(dge,design)
fit <- glmQLFit(y,design) 
qlf <- glmQLFTest(fit,coef=2)
qlf <- topTags(qlf, n=length(qlf$table$logFC))
FDRFilterQLF <- qlf$table$FDR <= 0.05
qlfWithFDRFilter <- qlf$table[FDRFilterQLF, ]


In [93]:
# Write quasi-likelihood F-tests with FDR > 0.05
write.csv(qlfWithFDRFilter, "/home/jupyter/qlfWithFDRFilter.csv")

In [7]:
#To perform likelihood ratio tests:
fit2 <- glmFit(y,design)
lrt <- glmLRT(fit2, coef=2)
lrt <- topTags(lrt, n=length(lr,t$table$logFC))
FDRFilterLTR <- lrt$table$FDR <= 0.05
lrtWithFDRFilter <- lrt$table[FDRFilterLTR, ]

ERROR: Error in topTags(lrt, n = length(lr, t$table$logFC)): object 'lr' not found


In [94]:
# Writing likelihood ratio tests with FDR > 0.05
write.csv(lrtWithFDRFilter, "/home/jupyter/lrtWithFDRFilter.csv")

In [8]:
# Analisys data with lima library
# Taking groups 
sex <- factor(groupsList$Sex)
Ethnicity <- factor(groupsList$Ethnicity)
Race <- factor(groupsList$Race)
SiteOnset <- factor(groupsList$SiteOnset)
age <- factor(groupsList$Age)
caseVscontrol = factor(groupsList$subjectGroup)

In [9]:
# Design matrix whit SiteOnset cases
design <- model.matrix(~0 + SiteOnset + Race + sex + age)
# Is necessary change the name of the columns becase have space between letters
colnames(design) <- c("SiteOnsetAxial","SiteOnsetBulbar","control","SiteOnsetLimb","SiteOnsetMultiple", "SiteOnsetOther", 
                     'RaceBlackAfricanAmerican', 'RaceBlackAfricanAmericanAmericanIndianAlaskaNative', 'RaceWhite',
                     'RaceWhiteAsian', 'RaceWhiteAmericanIndianAlaskaNative', 'sexMale', 'age2534', 'age3544', 'age4554',
                     'age5564', 'age65') 
# Fit quantile norm data with desgin matrix
fit_ALS <- lmFit(nor.data, design)
# Set contrast between two groups bulbar vs control
cont.matrix_ALS.bulbar_vs_control <- makeContrasts(SiteOnsetBulbar-control, levels = design)
fit_ALS_bulbar_vs_control <- contrasts.fit(fit_ALS,cont.matrix_ALS)
fit_ALS_bulbar_vs_control <- eBayes(fit_ALS_bulbar_vs_control)
# Set contrast between two groups bulbar vs limb
cont.matrix_ALS.bulbar_vs_limb <- makeContrasts(SiteOnsetBulbar-SiteOnsetLimb, levels = design)
fit_ALS_bulbar_vs_limb <- contrasts.fit(fit_ALS,cont.matrix_ALS)
fit_ALS_bulbar_vs_limb <- eBayes(fit_ALS_bulbar_vs_limb)

ERROR: Error in contrasts.fit(fit_ALS, cont.matrix_ALS): object 'cont.matrix_ALS' not found


In [ ]:
DEP_SIG_ALS_bulbar_vs_control <- topTable(fit_ALS_bulbar_vs_control, adjust = "BH", 
                                          number = length(fit_ALS_bulbar_vs_control$coefficients)) 
DEP_SIG_ALS_bulbar_vs_limb <- topTable(fit_ALS_bulbar_vs_limb, adjust = "BH", 
                                          number = length(fit_ALS_bulbar_vs_limb$coefficients)) 

In [ ]:
write.csv(DEP_SIG_ALS_bulbar_vs_control, 'SiteOnsetBulbar_vs_control.csv')
write.csv(DEP_SIG_ALS_bulbar_vs_limb, 'SiteOnsetBulbar_vs_limb.csv')

In [ ]:
# design new matrix with case vs controls
design <- model.matrix(~0 + caseVscontrol + Race + sex + age)
# Change the name of the columns
colnames(design) <- c("caseVscontrolALS","caseVscontrolHealthyControl","caseVscontrolNonALSMND", 
                     'RaceBlackAfricanAmerican', 'RaceBlackAfricanAmericanAmericanIndianAlaskaNative', 'RaceWhite',
                     'RaceWhiteAsian', 'RaceWhiteAmericanIndianAlaskaNative', 'sexMale', 'age2534', 'age3544', 'age4554',
                     'age5564', 'age65') 
# fit data
fit_case_control_ALS <- lmFit(nor.data, design)
# contrast case vs control data
cont.matrix_case_control <- makeContrasts(caseVscontrolALS-caseVscontrolHealthyControl, levels = design)
fit_case_control_ALS <- contrasts.fit(fit_case_control_ALS, cont.matrix_case_control)
fit_case_control_ALS <- eBayes(fit_case_control_ALS)

In [ ]:
DEP_SIG_ALS_case_vs_control <- topTable(fit_case_control_ALS, adjust = "BH", 
                                          number = length(fit_case_control_ALS$coefficients)) 

In [ ]:
write.csv(DEP_SIG_ALS_case_vs_control, 'ALS_case_vs_control.csv')